<a href="https://colab.research.google.com/github/ozturkosu/notes/blob/master/tensorizedMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os, sys, random
from sklearn.datasets import fetch_20newsgroups

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [3]:
newsgroups_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
len(newsgroups_train['data'])

11314

In [6]:
random.seed(1)

train_size = int(len(newsgroups_train['data'])*0.85)
dev_size = len(newsgroups_train['data']) - train_size

ng_train = {}
ng_train['data']= []
ng_train['target'] = []


ng_dev = {}
ng_dev['data'] = []
ng_dev['target'] = []


In [8]:
indices = list(range(len(newsgroups_train['data'])))
random.shuffle(indices)

train_idx = indices[:train_size]
dev_idx = indices[train_size:]

for i in train_idx:
    ng_train['data'].append(newsgroups_train['data'][i])
    ng_train['target'].append(newsgroups_train['target'][i])

for i in dev_idx:
    ng_dev['data'].append(newsgroups_train['data'][i])
    ng_dev['target'].append(newsgroups_train['target'][i])

#print(len(ng_dev['data']))


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)

In [54]:
train_x = vectorizer.fit_transform(ng_train['data'])


In [56]:
#target_x = vectorizer.fit_transform(ng_train['target'])

In [53]:
print('Shape of Train Data ', train_x.shape)

Shape of Train Data  (9616, 1000)


In [57]:
dev_x = vectorizer.transform(ng_dev['data'])

In [58]:
test_x = vectorizer.transform(newsgroups_test['data'])

In [19]:
print('Shape of Dev Data ', dev_x.shape)

Shape of Dev Data  (1698, 1000)


In [20]:
print('Shape of Test Data ', test_x.shape)

Shape of Test Data  (7532, 1000)


In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [67]:
class MLP(nn.Module):

  def __init__(self, r1=2, r2=2 , i=10 , j=10, k=10 , l=5 , m=10, n=2):
    super(MLP, self).__init__()

    self.T1=nn.Parameter(torch.rand(l,i,r1), requires_grad=True)
    self.T2=nn.Parameter(torch.rand(r1,m,j,r2), requires_grad=True)
    self.T3=nn.Parameter(torch.rand(n, k, r2), requires_grad=True)

    def forward(self, y):

    # Reshape y to make it 3D tensor from 1D tensor (vector)
      yt=y.view(10,10,10)

    # Calculate Tensor Contraction  P=T1y      T1y = T1*y # Note as index r2 represented in einsum as w #
      P=torch.einsum('lir,iyz->lryz', [self.T1, yt])

    # Calculate Tensor Contraction  Q=T1yT2   T1yT2 = (T1y) * T2
      Q=torch.einsum('lryz,rmyw->lzmw',[P,self.T2])

    # Calculate Tensor Contraction  R=T1yT2T3   T1yT2T3 = (T1yT2)* T3  #  k and z are same as indices
      R=torch.einsum('lzmw,nzw->lmn', [Q, self.T3]). # 3d Tensor with 2*5*10

    # Apply aktivation function
      result=F.log_softmax(R) 

      return result  



In [34]:
model= MLP()

In [68]:
model.train()

MLP()

In [ ]:
#Specify Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

#Specify Loss Function
criterion = nn.CrossEntropyLoss()



In [ ]:
n_epochs = 10
traindata_size = train_size
for epoch in range(n_epochs):
    #monitor training loss
    loss=0.0
    for i in range(traindata_size):
        
        optimizer.zero_grad()

        #Forward Pass: compute predicted outputs by passing inputs to the model
        output = model(train_x[i])

        #Calculate the loss
        if(output != target_x[i])
          loss+=1
        #loss = criterion(output, target_x[i])  # This is wrong since I do not know how to find target


        
        #Backward pass: compute gradient of the loss with respect to model parameters  

        loss.backward()
        optimizer.step()

    loss=loss/train_size
    print('For Epoch: {}, Loss:{}'.format(epoch, loss.item()))